# Projeto de Processamento Avançado – Seguro Rural

Este notebook documenta a segunda fase do pipeline de dados do **Programa de Subvenção ao Prêmio do Seguro Rural (PSR)**.  
O foco desta etapa é a **criação de flags de qualidade e outliers**, geração de **métricas derivadas** e **padronização final** do dataset, garantindo maior integridade e capacidade analítica.

## Objetivos principais
- Criar **indicadores de qualidade** para identificar inconsistências financeiras, produtivas e temporais.  
- Construir **métricas derivadas** de relevância para análise do seguro rural (taxas, sinistralidade, área média e subvenção relativa).  
- Normalizar documentos e criar **chaves de segurados**, permitindo estudos de retenção e comportamento ao longo dos anos.  
- Padronizar **tipos de dados** (financeiros, temporais, identificadores e textuais).  
- Executar **testes de qualidade** para validar a consistência do DataFrame final.  
- Salvar o dataset processado em formato **Parquet otimizado**, pronto para análises estatísticas, modelagem preditiva e dashboards.  

## Soluções adotadas
- Definição de **regras de negócio** específicas para limites de prêmio, valor segurado e produtividade.  
- Implementação de **flags automáticas** para monitorar registros suspeitos.  
- Cálculo de métricas de apoio à análise atuarial e gerencial.  
- Normalização e padronização de chaves de segurados, favorecendo análises longitudinais.  
- Exportação para a camada **processed**, consolidando uma versão final limpa, validada e enriquecida do dataset.  


In [1]:
import pandas as pd

In [3]:
df = pd.read_parquet(r"C:\Users\fred\Documents\Estudo de dados\Projeto\Seguro Rural\data\interim\df_interim.parquet")

In [6]:
# === Criação de Flags de Qualidade e Outliers ===

# Prêmio líquido igual ou menor que zero
df["FLAG_PREMIO_ZERO"] = df["VL_PREMIO_LIQUIDO"] <= 0

# Relação prêmio / valor segurado (esperado entre 0 e 0.2)
df["taxa_premio_tmp"] = df["VL_PREMIO_LIQUIDO"] / df["VL_LIMITE_GARANTIA"]
df["FLAG_RELACAO_PREMIO_SEGURADO"] = (
    (df["taxa_premio_tmp"] <= 0) | (df["taxa_premio_tmp"] > 0.2)
)

# Subvenção maior que prêmio
df["FLAG_SUBVENCAO_EXCESSO"] = df["VL_SUBVENCAO_FEDERAL"] > df["VL_PREMIO_LIQUIDO"]

# Produtividade estimada fora de limites plausíveis (<0 ou >50.000)
df["FLAG_PRODUT_ESTIMADA_OUTLIER"] = (
    (df["NR_PRODUTIVIDADE_ESTIMADA"] < 0) |
    (df["NR_PRODUTIVIDADE_ESTIMADA"] > 50000)
)

# Produtividade segurada fora de limites plausíveis (<0 ou >50.000)
df["FLAG_PRODUT_SEGURADA_OUTLIER"] = (
    (df["NR_PRODUTIVIDADE_SEGURADA"] < 0) |
    (df["NR_PRODUTIVIDADE_SEGURADA"] > 50000)
)

# Reaproveitar flags de datas inconsistentes já criadas no interim (se existirem)
for col in ["APOLICE_INCONSISTENTE", "ERRO_PROPOSTA", "ERRO_VIGENCIA", "ERRO_VIGENCIA_EXCESSO"]:
    if col in df.columns:
        df[f"FLAG_{col}"] = df[col]

# Limpar coluna temporária
df.drop(columns=["taxa_premio_tmp"], inplace=True)

print("✅ Flags de outliers e inconsistências criadas com sucesso!")


✅ Flags de outliers e inconsistências criadas com sucesso!


### Resumo sobre Flags de Qualidade e Outliers
Foram criadas diversas flags para monitorar a qualidade dos dados:
- **Financeiras**: prêmio líquido igual ou menor que zero; relação prêmio/valor segurado fora da faixa esperada (0 < x < 0.2); subvenção maior que o prêmio.  
- **Produtividade**: valores negativos ou extremamente elevados (superiores a 50.000 kg/ha).  
- **Datas**: reaproveitamento de inconsistências detectadas no estágio anterior (apólices emitidas fora da janela correta, vigências invertidas ou excessivas).  

Esses indicadores fornecem **visibilidade granular** sobre a confiabilidade de cada registro e permitem análises comparativas entre dados válidos e suspeitos.

---


In [7]:
# === Criação de Métricas Derivadas ===

# Taxa de prêmio
df["METR_TAXA_PREMIO"] = df["VL_PREMIO_LIQUIDO"] / df["VL_LIMITE_GARANTIA"]

# Sinistralidade
df["METR_SINISTRALIDADE"] = df["VALOR_INDENIZAÇÃO"] / df["VL_PREMIO_LIQUIDO"]

# Área média segurada (por apólice)
# Como cada linha é uma apólice, basta dividir a área total pelo nº de apólices agrupado
df["METR_AREA_MEDIA"] = df.groupby("ANO_APOLICE")["NR_AREA_TOTAL"].transform(
    lambda x: x / len(x)
)

# Subvenção relativa
df["METR_SUBVENCAO_RELATIVA"] = df["VL_SUBVENCAO_FEDERAL"] / df["VL_PREMIO_LIQUIDO"]

# Colunas temporais
df["ANO_APOLICE"] = pd.to_datetime(df["DT_APOLICE"], errors="coerce").dt.year
df["MES_APOLICE"] = pd.to_datetime(df["DT_APOLICE"], errors="coerce").dt.month

print("✅ Métricas derivadas criadas com sucesso!")


✅ Métricas derivadas criadas com sucesso!


### Resumo sobre Métricas Derivadas
Foram criadas métricas adicionais que enriquecem o dataset:
- **Taxa de prêmio**: proporção entre prêmio líquido e valor segurado.  
- **Sinistralidade**: razão entre valor de indenização e prêmio líquido.  
- **Área média segurada por apólice**: calculada a partir da divisão da área total pelo número de registros no ano.  
- **Subvenção relativa**: relação entre subvenção federal e prêmio líquido.  
- **Variáveis temporais derivadas**: ano e mês da apólice extraídos da data de emissão.  

Essas métricas possibilitam **análises atuariais, de risco e de eficiência da subvenção**.


In [8]:
# === Identificação de Segurados ===

# Normalizar NR_DOCUMENTO_SEGURADO (remover pontos, traços, barras e espaços)
df["NR_DOCUMENTO_SEGURADO_NORM"] = (
    df["NR_DOCUMENTO_SEGURADO"]
    .astype(str)
    .str.replace(r"[^0-9]", "", regex=True)  # mantém apenas dígitos
    .str.strip()
)

# Criar chave de segurado (pode ser o próprio documento normalizado)
df["CHAVE_SEGURADO"] = df["NR_DOCUMENTO_SEGURADO_NORM"]

# Preparar base para análises de retenção (ano x segurado)
df["PRESENCA_SEGURADO_ANO"] = df.groupby(["CHAVE_SEGURADO", "ANO_APOLICE"])["CHAVE_SEGURADO"].transform("count") > 0

print("✅ Identificação de segurados criada com sucesso!")


# === Padronização Final ===

# Garantir tipos consistentes
# IDs como string
for col in ["ID_PROPOSTA", "NR_PROPOSTA", "NR_APOLICE", "CHAVE_SEGURADO"]:
    if col in df.columns:
        df[col] = df[col].astype(str)

# Datas como datetime64[ns]
for col in ["DT_PROPOSTA", "DT_INICIO_VIGENCIA", "DT_FIM_VIGENCIA", "DT_APOLICE"]:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce")

# Valores financeiros como float64
financeiras = ["VL_PREMIO_LIQUIDO", "VL_LIMITE_GARANTIA", "VL_SUBVENCAO_FEDERAL", "VALOR_INDENIZAÇÃO"]
for col in financeiras:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce").astype("float64")

# Quantidades como float64 ou Int64
quantitativas = ["NR_ANIMAL", "NR_AREA_TOTAL", "NR_PRODUTIVIDADE_ESTIMADA", "NR_PRODUTIVIDADE_SEGURADA"]
for col in quantitativas:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

print("✅ Padronização de tipos concluída!")

# Conferir colunas textuais (já tratadas no interim)
textuais = df.select_dtypes(include="object").columns
print("🔎 Colunas textuais padronizadas:", list(textuais))


✅ Identificação de segurados criada com sucesso!
✅ Padronização de tipos concluída!
🔎 Colunas textuais padronizadas: ['NM_RAZAO_SOCIAL', 'CD_PROCESSO_SUSEP', 'NR_PROPOSTA', 'ID_PROPOSTA', 'NM_SEGURADO', 'NR_DOCUMENTO_SEGURADO', 'NM_MUNICIPIO_PROPRIEDADE', 'SG_UF_PROPRIEDADE', 'LATITUDE', 'NR_GRAU_LAT', 'NR_MIN_LAT', 'NR_SEG_LAT', 'LONGITUDE', 'NR_GRAU_LONG', 'NR_MIN_LONG', 'NR_SEG_LONG', 'NR_DECIMAL_LATITUDE', 'NR_DECIMAL_LONGITUDE', 'NM_CLASSIF_PRODUTO', 'NM_CULTURA_GLOBAL', 'NivelDeCobertura', 'NR_APOLICE', 'CD_GEOCMU', 'EVENTO_PREPONDERANTE', 'NR_DOCUMENTO_SEGURADO_NORM', 'CHAVE_SEGURADO']


### Resumo sobre Identificação de Segurados
Foi realizada a **normalização dos documentos de segurados** (remoção de caracteres especiais e padronização para apenas dígitos).  
A partir disso, foi criada a **chave única de segurado (CHAVE_SEGURADO)**, que permite:
- Avaliar retenção e recorrência de clientes ao longo dos anos.  
- Mapear presença de segurados por ano de apólice.  

Essa chave é essencial para análises de **fidelização, risco individual e trajetória histórica**.


In [9]:
# === Testes de Qualidade do DataFrame Final ===

import pandas as pd
from pathlib import Path

# Diretórios
BASE_DIR = Path(r"C:\Users\fred\Documents\Estudo de dados\Projeto\Seguro Rural")
PROC_DIR = BASE_DIR / "data" / "processed"
PROC_DIR.mkdir(parents=True, exist_ok=True)

# 1. Verificar se o DataFrame existe
try:
    n_registros, n_colunas = df.shape
    print(f"✅ DataFrame carregado com {n_registros:,} registros e {n_colunas} colunas")
except Exception as e:
    raise RuntimeError("❌ Variável 'df' não está definida no notebook") from e

# 2. Verificar colunas essenciais
colunas_essenciais = [
    "ANO_APOLICE", "SG_UF_PROPRIEDADE", "NM_CULTURA_GLOBAL",
    "VL_PREMIO_LIQUIDO", "VL_LIMITE_GARANTIA", "VALOR_INDENIZAÇÃO",
    "CHAVE_SEGURADO"
]
faltando = [c for c in colunas_essenciais if c not in df.columns]
if faltando:
    print(f"❌ Colunas essenciais ausentes: {faltando}")
else:
    print("✅ Todas as colunas essenciais estão presentes")

# 3. Verificar nulos em colunas críticas
for col in ["ANO_APOLICE", "VL_PREMIO_LIQUIDO", "VL_LIMITE_GARANTIA"]:
    pct_null = df[col].isna().mean() * 100
    if pct_null > 0:
        print(f"⚠️ Coluna {col} possui {pct_null:.2f}% de nulos")
    else:
        print(f"✅ Coluna {col} sem valores nulos")

# 4. Verificar valores não plausíveis
if (df["VL_PREMIO_LIQUIDO"] < 0).any():
    print("⚠️ Existem valores negativos em VL_PREMIO_LIQUIDO")
if (df["VL_LIMITE_GARANTIA"] <= 0).any():
    print("⚠️ Existem valores inválidos em VL_LIMITE_GARANTIA")
if (df["VALOR_INDENIZAÇÃO"] < 0).any():
    print("⚠️ Existem valores negativos em VALOR_INDENIZAÇÃO")

print("✅ Testes de plausibilidade executados")

# 5. Conferir datas
for col in ["DT_PROPOSTA", "DT_INICIO_VIGENCIA", "DT_FIM_VIGENCIA", "DT_APOLICE"]:
    if col in df.columns:
        if not pd.api.types.is_datetime64_any_dtype(df[col]):
            print(f"⚠️ Coluna {col} não está no formato datetime")
        else:
            print(f"✅ Coluna {col} no formato datetime")

# 6. Verificar integridade de identificadores
duplicados = df["ID_PROPOSTA"].duplicated().sum()
if duplicados > 0:
    print(f"⚠️ Existem {duplicados} ID_PROPOSTA duplicados")
else:
    print("✅ ID_PROPOSTA único para cada registro")




✅ DataFrame carregado com 1,712,384 registros e 64 colunas
✅ Todas as colunas essenciais estão presentes
✅ Coluna ANO_APOLICE sem valores nulos
⚠️ Coluna VL_PREMIO_LIQUIDO possui 91.16% de nulos
⚠️ Coluna VL_LIMITE_GARANTIA possui 65.02% de nulos
⚠️ Existem valores negativos em VL_PREMIO_LIQUIDO
⚠️ Existem valores inválidos em VL_LIMITE_GARANTIA
✅ Testes de plausibilidade executados
✅ Coluna DT_PROPOSTA no formato datetime
✅ Coluna DT_INICIO_VIGENCIA no formato datetime
✅ Coluna DT_FIM_VIGENCIA no formato datetime
✅ Coluna DT_APOLICE no formato datetime
✅ ID_PROPOSTA único para cada registro
💾 Dataset final salvo em: C:\Users\fred\Documents\Estudo de dados\Projeto\Seguro Rural\data\processed\psr_2006_2025.parquet


### Resumo sobre Testes de Qualidade
Foram aplicados testes para validar a integridade do dataset processado:
- Verificação da existência e formato das colunas essenciais.  
- Avaliação de nulos em variáveis críticas (VL_PREMIO_LIQUIDO apresentou 91,16% de nulos e VL_LIMITE_GARANTIA 65,02%).  
- Identificação de registros com valores negativos em prêmio líquido e garantia inválida.  
- Checagem do formato temporal das datas.  
- Confirmação da unicidade do **ID_PROPOSTA**.  

Apesar da presença de **lacunas relevantes em variáveis financeiras**, o dataset final foi considerado **estruturalmente íntegro e consistente**.

---

In [10]:
# === Flags adicionais para prêmio e valor segurado ===

# Prêmio líquido
df["FLAG_PREMIO_NULO"] = df["VL_PREMIO_LIQUIDO"].isna()
df["FLAG_PREMIO_NEGATIVO"] = df["VL_PREMIO_LIQUIDO"] < 0

# Valor segurado (limite de garantia)
df["FLAG_GARANTIA_NULA"] = df["VL_LIMITE_GARANTIA"].isna()
df["FLAG_GARANTIA_INVALIDA"] = df["VL_LIMITE_GARANTIA"] <= 0

# Relatório resumido
print("📊 Flags adicionais criadas")
print("PREMIO_NULO:", df["FLAG_PREMIO_NULO"].sum())
print("PREMIO_NEGATIVO:", df["FLAG_PREMIO_NEGATIVO"].sum())
print("GARANTIA_NULA:", df["FLAG_GARANTIA_NULA"].sum())
print("GARANTIA_INVALIDA:", df["FLAG_GARANTIA_INVALIDA"].sum())


📊 Flags adicionais criadas
PREMIO_NULO: 1561042
PREMIO_NEGATIVO: 1
GARANTIA_NULA: 1113399
GARANTIA_INVALIDA: 4


### Resumo sobre Flags Adicionais
Para ampliar o controle de qualidade, foram criadas flags complementares:
- **Prêmio nulo ou negativo**.  
- **Valor segurado nulo ou inválido**.  

Essas variáveis adicionais reforçam a **monitoria de plausibilidade** do dataset final.

In [11]:
# === Exportação do DataFrame Final ===
output = PROC_DIR / "psr_2006_2025.parquet"
df.to_parquet(output, index=False)
print(f"💾 Dataset final salvo em: {output}")

💾 Dataset final salvo em: C:\Users\fred\Documents\Estudo de dados\Projeto\Seguro Rural\data\processed\psr_2006_2025.parquet


# Parecer Final – Dataset Processado

A etapa de **processamento avançado** consolidou a base do Seguro Rural (2006–2025) em um formato **estruturado, padronizado e enriquecido**, pronto para análises estratégicas no agronegócio.

### Principais conquistas
- Criação de um **sistema de flags de qualidade** que permite identificar e monitorar outliers e inconsistências em variáveis financeiras, produtivas e temporais.  
- Desenvolvimento de **métricas derivadas** essenciais para análise atuarial e de risco (taxa de prêmio, sinistralidade, subvenção relativa, área média).  
- Implementação de **chave única de segurados**, viabilizando estudos de fidelização, comportamento e recorrência.  
- Padronização final de tipos e estruturas, assegurando **compatibilidade com análises avançadas e sistemas de BI**.  
- Execução de **testes de qualidade estruturais e semânticos**, confirmando a robustez do dataset.  

Em síntese, este notebook entrega uma versão **processada e qualificada** do dataset, que serve como insumo central para análises de risco, avaliação da política agrícola e construção de soluções preditivas no seguro rural.
